# Example Illustration

To use `a2` in a project:

In [1]:
import a2

print(a2.__version__)

0.1.0


## Exercise1 from [page 59 of Algorithms and Data Structures for Massive Datasets](https://ebookcentral-proquest-com.proxy.library.georgetown.edu/lib/georgetown/reader.action?docID=7049417&ppg=64).

Key Requirements:
Bloom Filter Setup:
- n = 10 ^ 7
- f = 0.02 (target false positive rate = 2%)
- Elements should be uniform random integers in the range [0, 10^12], converted into strings.

Tasks:
- Save the inserted elements into a file.
- Perform 10^6 (1 million) lookups using randomly selected elements from the universe U in the range [0, 10^12].
- Perform 10^6 (1 million) successful lookups using elements from the inserted elements.
- Track the false positive rate and verify it is close to 2%.

Time Measurement:
- Measure the time required for the lookups, but exclude the time for random number generation.

In [2]:
import math 
import mmh3
import pandas as pd
import numpy as np
from bitarray import bitarray
# exercise 1
import random
import time
from a2.bloomfilter1 import BloomFilter
import zipfile

### Initialize bloom filter with n and f

In [3]:
n = 10**7
f = 0.02
bf = BloomFilter(n, f)


Init parameters:
n = 10000000, f=0.02, m=81423633, k=5


Randomly generate integer, convert to string, save to file


In [4]:
with open('../src/a2/inserted_elements.txt', 'w') as f:
    for n in range(n): 
        random_integer = random.randint(0, 10**12)
        random_string = str(random_integer)
        bf.insert(random_string)
        # Write the inserted element to the file
        f.write(f"{random_string}\n")

In [5]:
lo_elements = []
with open('../src/a2/inserted_elements.txt', 'r') as f:
    lo_elements = [line.strip() for line in f]

print(lo_elements[:5])

['100963958600', '254970694481', '259722675981', '438222137969', '700849256904']


### Uniform Random Lookup:
Elements used: randomly generate elements from U, elements are not necessarily in the Bloom filter. 

In [6]:
looks = 10 ** 6
real_uniform_f = 0 # false positive rate in simulation
# generate integer, convert to string
lookup_elements = [str(random.randint(0, 10**12)) for l in range(looks)]
start_time = time.time()
for j in range(looks):
    current_element = lookup_elements[j]
    if bf.query(current_element):
        real_uniform_f += 1

end_time = time.time()
print("Required time for performing 10^6 lookups of uniformly randomly selected elements from U: {} seconds".format(round(end_time-start_time, 3)))
print("False Positive rate during look-up is: {}%".format(real_uniform_f / looks * 100))


Required time for performing 10^6 lookups of uniformly randomly selected elements from U: 0.327 seconds
False Positive rate during look-up is: 2.0303999999999998%



### Successful lookups 
Perform lookups using elements that were actually inserted into the Bloom filter. Selecting these elements from the file of inserted elements (inserted_elements.txt), ensuring that each lookup will be successful 


In [7]:
real_successful_r = 0 # false positive rate in successful lookups
# generate integer, convert to string
start_time2 = time.time()
for z in range(looks):
    current_element = random.choice(lo_elements)
    if bf.query(current_element):
        real_successful_r += 1

end_time2 = time.time()
print("Required time for performing 10^6 successful lookups: {} seconds".format(round(end_time2-start_time2, 3)))
print("Rate of successfylly find element in Bloom Filter during successful look-up is: {}%".format(real_successful_r / looks * 100))

Required time for performing 10^6 successful lookups: 1.115 seconds
Rate of successfylly find element in Bloom Filter during successful look-up is: 100.0%


## EXERCISE 2 from [page 59 of Algorithms and Data Structures for Massive Datasets](https://ebookcentral-proquest-com.proxy.library.georgetown.edu/lib/georgetown/reader.action?docID=7049417&ppg=64).
Using the provided implementation, create a Bloom filter such as the one in Example 2. Now create two other filters, one in which the dataset is 100 times larger than the original one, and another one in which the dataset is 100 times smaller, leaving the same false positive rate. What do you notice about the size of the filter as the dataset size changes?

Key requirements from Example 2:
- n = 10 ^ 6 elements
- 1 MB available for storing (m = 8 * 10^6 bits)

Tasks: 
- Construct a Bloom Filter whose dataset is 100 times larger than the original one(the one we created for example2)
- Construct a Bloom Filter whose dataset is 100 times smaller than the original one(the one we created for example2)
- Share findings of the size of the filter as the dataset size changes

In [8]:
def calculateF(k):
    """calculate false positive rate based on given k
    return float: the value of f
    """
    return (0.5) ** k

In [9]:
# Initialize a new filter based on parameters provided in example2
n2 = 10**6
m2 = 8 * 10**6
f2 = 0
bf2 = BloomFilter(n=n2, f=f2, m=m2)
bf2.f = calculateF(bf2.k)
bf2.printParameters()

Init parameters:
n = 1000000, f=0, m=8000000, k=5
Init parameters:
n = 1000000, f=0.03125, m=8000000, k=5


In [10]:
bf3 = BloomFilter(n = 100*bf2.n, f = bf2.f)

Init parameters:
n = 100000000, f=0.03125, m=721347520, k=4


In [11]:
bf4 = BloomFilter(n = 0.01*bf2.n, f = bf2.f)

Init parameters:
n = 10000.0, f=0.03125, m=72134, k=4


In [12]:
print("Parameters of Bloom Filter where the dataset is 100 times larger than the original one: ")
bf3.printParameters()
print("Parameters of Bloom Filter where the dataset is 100 times smaller than the original one: ")
bf4.printParameters()
print("Ratio of the sizes between two new filters: ", round(bf3.m/bf4.m))

Parameters of Bloom Filter where the dataset is 100 times larger than the original one: 
Init parameters:
n = 100000000, f=0.03125, m=721347520, k=4
Parameters of Bloom Filter where the dataset is 100 times smaller than the original one: 
Init parameters:
n = 10000.0, f=0.03125, m=72134, k=4
Ratio of the sizes between two new filters:  10000


***Findings: What we found from increasing the size of dataset of Bloom Filter is that: as we increase the dataset size by 100 time, the number of bits in the Bloom Filter, or the size of filter, increases 100 times as well. As we shrink the dataset size by 100 times, the number of bits in the Bloom Filter/size of filter also decreases.***

## EXERCISE 3 from [page 59 of Algorithms and Data Structures for Massive Datasets](https://ebookcentral-proquest-com.proxy.library.georgetown.edu/lib/georgetown/reader.action?docID=7049417&ppg=64).

In some literature, a variant of a Bloom filter is described where different hash functions have the “jurisdiction” over different parts of the Bloom filter. In other words,  k  hash functions split the Bloom filter into  k  equal-sized consecutive chunks of  m/k  bits, and during an insert, the  i th hash function sets bits in the  i th chunk. Implement this variant of the Bloom filter and check if and how this change might affect the false positive rate in comparison to the original Bloom filter.

Key requirements from Example 3:
- n = len(lines) # the length of the tsv file
- p = 0.2
- for chunked bloom filter, it has 5 parts.

Tasks: 
- Designed a chunked Bloom filter.
- Compare the false positive rate of the chunked Bloom filter with that of the original one.
- Make the conclusions.

In [13]:
import zipfile
import hashlib
from bitarray import bitarray
import re
import math
from a2.bloomfilter3 import StandardBloomFilter, ChunkedBloomFilter, ImprovedBloomFilter

In [14]:
def normalize_text(text):
    text = text.lower() 
    text = re.sub(r'\s+', ' ', text) 
    text = re.sub(r'[^\w\s]', '', text) 
    return text

def tokenize_text(line):
    parts = line.split('\t')
    tokens = []
    for part in parts:
        tokens.extend(part.split())
    return tokens

def calculate_bloom_filter_size(n, p):
    m = - (n * math.log(p)) / (math.log(2) ** 2)
    k = (m / n) * math.log(2)
    return int(m), int(k)

In [15]:
zip_path = '../duplicates.zip'
required_files = ['duplicates/threehundred.tsv', 'duplicates/onek.tsv', 'duplicates/tenk.tsv', 'duplicates/hundredk.tsv']
password = b'123456'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_files = zip_ref.namelist()

    for file in required_files:
        if file in zip_files:
            print(f"File: {file}")
            with zip_ref.open(file, pwd=password) as f:
                lines = [line.decode('utf-8').strip() for line in f.readlines()]

                n = len(lines)
                p = 0.2
                m, k = calculate_bloom_filter_size(n, p)

                standard_bloom = StandardBloomFilter(m, k)
                chunked_bloom = ChunkedBloomFilter(m, k, 5)

                false_positive_standard = 0
                false_positive_chunked = 0

                elements_added = int(len(lines) * 0.20)
                added_elements = lines[:elements_added]
                test_elements = lines[elements_added:]

                for line in added_elements:
                    standard_bloom.add(line)
                    chunked_bloom.add(line)

                for line in test_elements:
                    if standard_bloom.check(line):
                        false_positive_standard += 1
                    if chunked_bloom.check(line):
                        false_positive_chunked += 1

                if len(test_elements) > 0:
                    print(f"Standard Bloom Filter False Positive Rate: {false_positive_standard / len(test_elements):.4f}")
                    print(f"Chunked Bloom Filter False Positive Rate: {false_positive_chunked / len(test_elements):.4f}")
        else:
            print(f"File {file} not found in ZIP.")

File: duplicates/threehundred.tsv
Standard Bloom Filter False Positive Rate: 0.0042
Chunked Bloom Filter False Positive Rate: 0.0583
File: duplicates/onek.tsv
Standard Bloom Filter False Positive Rate: 0.0138
Chunked Bloom Filter False Positive Rate: 0.0700
File: duplicates/tenk.tsv
Standard Bloom Filter False Positive Rate: 0.0145
Chunked Bloom Filter False Positive Rate: 0.0648
File: duplicates/hundredk.tsv
Standard Bloom Filter False Positive Rate: 0.0122
Chunked Bloom Filter False Positive Rate: 0.0661


***Findings: For all four TSV files, we can see that the false positive rate of the chunked Bloom filter is always higher than that of the standard Bloom filter. This indicates that with the part number set to 5, the performance of the original Bloom filter is better on these TSV files. The results might change if the part number is altered.***

# EXERCISE 3 - Addition from Assignment2

Repeat exercise 3 by making two changes to your implementation of the Bloom filter in bloom_filter.py. How does the false positive rate change now? Specifically, explore three improvements of your choice to the Bloom filter. This is intentionally open-ended, and I’m excited to see what you experiment with. It is perfectly fine if you explore choices that do not lead to any substantial benefits. Improvements to Bloom filters are discussed in Algorithms and Data Structures for Massive Datasets, or you can find other materials online and cite them in your discussion. An example: the Kirsch-Mitzenmacher-Optimization computes two hashes instead of k hash functions. Here’s one more example: Hugging Face’s DataTrove has deduplication code that uses universal hashing to come up with k independent hash functions. We chose hash functions differently. Maybe you can try their approach. Or using a quotient filter is another option, as discussed in Algorithms and Data Structures for Massive Datasets. There are lots of options available to you. The most important point: think of your team as running experiments, exploring alternatives, and presenting your results to a stakeholder in the project.

Key requirements from Example 3 - Addition:
- n = len(lines) # the length of the tsv file
- p = 0.2
- for chunked bloom filter, it has 5 parts.
- for improved bloom filter, the method here is kirsch-mitzenmacher

Tasks: 
- Designed a improved Bloom filter with kirsch-mitzenmacher method.
- Compare the false positive rate of the improved Bloom filter with that of the chunked and original one.
- Make the conclusions.

In [16]:
zip_path = '../duplicates.zip'
required_files = ['duplicates/threehundred.tsv', 'duplicates/onek.tsv', 'duplicates/tenk.tsv', 'duplicates/hundredk.tsv']
password = b'123456'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_files = zip_ref.namelist()

    for file in required_files:
        if file in zip_files:
            print(f"File: {file}")
            with zip_ref.open(file, pwd=password) as f:
                lines = [line.decode('utf-8').strip() for line in f.readlines()]

                n = len(lines)
                p = 0.2
                m, k = calculate_bloom_filter_size(n, p)

                standard_bloom = StandardBloomFilter(m, k)
                chunked_bloom = ChunkedBloomFilter(m, k, 5)
                improved_bloom = ImprovedBloomFilter(m, k, method='kirsch-mitzenmacher')

                false_positive_standard = 0
                false_positive_chunked = 0
                false_positive_improved = 0

                elements_added = int(len(lines) * 0.20)
                added_elements = lines[:elements_added]
                test_elements = lines[elements_added:]

                for line in added_elements:
                    standard_bloom.add(line)
                    chunked_bloom.add(line)
                    improved_bloom.add(line)

                for line in test_elements:
                    if standard_bloom.check(line):
                        false_positive_standard += 1
                    if chunked_bloom.check(line):
                        false_positive_chunked += 1
                    if improved_bloom.check(line):
                        false_positive_improved += 1

                if len(test_elements) > 0:
                    print(f"Standard Bloom Filter False Positive Rate: {false_positive_standard / len(test_elements):.4f}")
                    print(f"Chunked Bloom Filter False Positive Rate: {false_positive_chunked / len(test_elements):.4f}")
                    print(f"Improved Bloom Filter False Positive Rate: {false_positive_improved / len(test_elements):.4f}")
        else:
            print(f"File {file} not found in ZIP.")

File: duplicates/threehundred.tsv
Standard Bloom Filter False Positive Rate: 0.0042
Chunked Bloom Filter False Positive Rate: 0.0583
Improved Bloom Filter False Positive Rate: 0.0125
File: duplicates/onek.tsv
Standard Bloom Filter False Positive Rate: 0.0138
Chunked Bloom Filter False Positive Rate: 0.0700
Improved Bloom Filter False Positive Rate: 0.0150
File: duplicates/tenk.tsv
Standard Bloom Filter False Positive Rate: 0.0145
Chunked Bloom Filter False Positive Rate: 0.0648
Improved Bloom Filter False Positive Rate: 0.0111
File: duplicates/hundredk.tsv
Standard Bloom Filter False Positive Rate: 0.0122
Chunked Bloom Filter False Positive Rate: 0.0661
Improved Bloom Filter False Positive Rate: 0.0129


***Findings: Comparing the results of four TSV files, we find that while the chunked Bloom filter always underperforms compared to the standard and improved Bloom filters, the performance between the standard and improved versions is uncertain. For the 300 and 1000 TSV files, the false positive rate of the improved Bloom filter is higher, but the situation changes with the 10k TSV file. Also, with the 100k TSV file, the performance of the two is almost identical. As parameters change, the results may also vary.***